<a href="https://colab.research.google.com/github/kaichiii/taitanic/blob/main/%E3%82%B5%E3%83%B3%E3%82%B7%E3%83%A3%E3%82%A4%E3%83%B3%E3%81%99%E3%81%B7%E3%81%97%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 実績精度集計マスタからコピーして作ったスプシのURL
ss_url = "https://docs.google.com/spreadsheets/d/1CW28F8V0u0umYWbA4F4mgRL_oVC_INPB-BslNxMX-w8/edit#gid=133542358"

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive 
drive.mount('/content/drive/')

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
workbook = gc.open_by_url(ss_url)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [18]:
from statistics import stdev, mean
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from google.cloud import bigquery

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [4]:
def write(df, sheet_title):
  master_worksheet = workbook.worksheet("日別")

  sheet = workbook.duplicate_sheet(source_sheet_id=master_worksheet.id, new_sheet_name=sheet_title, insert_sheet_index=1)
  sheet.add_rows(df.shape[0])

  for row_index, row in df.iterrows():
    row_index = row_index + 3
    cell_list = sheet.range(row_index, 2, row_index, sheet.col_count)

    target_row = []
    target_row.append(row['sales_date'])
    target_row.append(row['sum_sales_amount'])
    target_row.append(row['sum_sales_qty'])
    target_row.append(row['cnt_store'])
    target_row.append(row['cnt_sku'])

    for idx, data in enumerate(target_row):
      cell_list[idx].value = data

    res = sheet.update_cells(cell_list, value_input_option='USER_ENTERED')

    



In [5]:
project_id = 'sails-1980'
bq_client = bigquery.Client(project_id)

# 従来vs実験の精度集計

In [6]:
query = f'''
select
  sales_date,
  sum(cast(sales_amount as int64)) as sum_sales_amount,
  sum(cast(sales_qty as int64)) as sum_sales_qty,
  count(distinct store_id) as unique_store,
  count(distinct sku_id) as unique_sku
from `sails-1980.sunshine_data_check.sales_transaction_v000_20230412`
group by 
  sales_date
'''

metric_df = bq_client.query(query).to_dataframe()

print(metric_df.shape)
metric_df

(11, 5)


,sales_date,sum_sales_amount,sum_sales_qty,cnt_store,cnt_sku
0,20220401,99203748,412872,29,24043
1,20220402,91116736,355525,29,23748
2,20220403,157970345,513578,29,25552
3,20220404,74171635,312699,28,22310
4,20220405,74898050,304486,29,22472
5,20220406,97017984,372894,29,23793
6,20220407,74564937,313178,29,22514
7,20220408,86103158,338810,29,23147
8,20220409,115785335,393652,29,24328
9,20220410,140260084,474611,29,25250


In [8]:
sheet_title = '日別4'
write(metric_df, sheet_title)

In [ ]:
# def analysis(df,sheet_title):
#   worksheet = workbook.worksheet(sheet_title)
#   # C13を変えたい
#   sum_amount=0
#   for i in range(df.shape[0]):
#     value=int(worksheet.cell(i+3,3).value)
#     sum_amount+=value
#   print(sum_amount/df.shape[0])
  
  sum_qty=0
  for j in range(df.shape[0]):
    value=int(worksheet.cell(j+3,4).value)
    sum_qty+=value
  print(sum_qty/df.shape[0])


  

In [49]:
def analysis(df,sheet_title):
  worksheet = workbook.worksheet(sheet_title)
  sum_amount=[]
  for i in range(df.shape[0]):
    value=int(worksheet.cell(i+3,3).value)
    sum_amount.append(value)
  #mean_amountは平均,stdev_amountは標準偏差
  mean_amount=int(mean(sum_amount))
  stdev_amount=int(stdev(sum_amount))
  stdev_amount_2=int(stdev(sum_amount))*2
  stdev_amount_3=int(stdev(sum_amount))*3
  sigma_plus_2_amount=mean_amount+stdev_amount_2
  sigma_minus_2_amount=mean_amount-stdev_amount_2
  sigma_plus_3_amount=mean_amount+stdev_amount_3
  sigma_minus_3_amount=mean_amount-stdev_amount_3

  sum_qty=[]
  for j in range(df.shape[0]):
    value=int(worksheet.cell(j+3,4).value)
    sum_qty.append(value)
  #mean_amountは平均,stdev_amountは標準偏差
  mean_qty=int(mean(sum_qty))
  stdev_qty=int(stdev(sum_qty))
  stdev_qty_2=int(stdev(sum_qty))*2
  stdev_qty_3=int(stdev(sum_qty))*3
  sigma_plus_2_qty=mean_qty+stdev_qty_2
  sigma_minus_2_qty=mean_qty-stdev_qty_2
  sigma_plus_3_qty=mean_qty+stdev_qty_3
  sigma_minus_3_qty=mean_qty-stdev_qty_3

  sum_store=[]
  for k in range(df.shape[0]):
    value=int(worksheet.cell(j+3,4).value)
    sum_store.append(value)
  #mean_amountは平均,stdev_amountは標準偏差
  mean_store=int(mean(sum_store))
  stdev_store=int(stdev(sum_store))
  stdev_store_2=int(stdev(sum_store))*2
  stdev_store_3=int(stdev(sum_store))*3
  sigma_plus_2_store=mean_store+stdev_store_2
  sigma_minus_2_store=mean_store-stdev_store_2
  sigma_plus_3_store=mean_store+stdev_store_3
  sigma_minus_3_store=mean_store-stdev_store_3

  sum_sku=[]
  for l in range(df.shape[0]):
    value=int(worksheet.cell(j+3,4).value)
    sum_sku.append(value)
  #mean_amountは平均,stdev_amountは標準偏差
  mean_sku=int(mean(sum_sku))
  stdev_sku=int(stdev(sum_sku))
  stdev_sku_2=int(stdev(sum_sku))*2
  stdev_sku_3=int(stdev(sum_sku))*3
  sigma_plus_2_sku=mean_sku+stdev_sku_2
  sigma_minus_2_sku=mean_sku-stdev_sku_2
  sigma_plus_3_sku=mean_sku+stdev_sku_3
  sigma_minus_3_sku=mean_sku-stdev_sku_3


  
  list_name=['average', 'Standard Deviation(σ)','2σ','3σ','average+2σ','average-2σ','average+3σ','average-3σ']
  list_amount=[mean_amount, stdev_amount, stdev_amount_2, stdev_amount_3, sigma_plus_2_amount, sigma_minus_2_amount, sigma_plus_3_amount, sigma_minus_3_amount]
  list_qty=[mean_qty, stdev_qty, stdev_qty_2, stdev_qty_3, sigma_plus_2_qty, sigma_minus_2_qty, sigma_plus_3_qty, sigma_minus_3_qty]
  list_store=[mean_store, stdev_store, stdev_store_2, stdev_store_3, sigma_plus_2_store, sigma_minus_2_store, sigma_plus_3_store, sigma_minus_3_store]
  list_sku=[mean_sku, stdev_sku, stdev_sku_2, stdev_sku_3, sigma_plus_2_sku, sigma_minus_2_sku, sigma_plus_3_sku, sigma_minus_3_sku]



  for x in range(len(list_name)):
    worksheet.update_cell(4+df.shape[0]+x, 1, list_name[x])

  for y in range(len(list_amount)):
    worksheet.update_cell(4+df.shape[0]+y, 3, list_amount[y])

  for z in range(len(list_qty)):
    worksheet.update_cell(4+df.shape[0]+z, 4, list_qty[z])

  for a in range(len(list_store)):
    worksheet.update_cell(4+df.shape[0]+a, 5, list_qty[a])

  for b in range(len(list_sku)):
    worksheet.update_cell(4+df.shape[0]+b,6, list_qty[b])




  

  

  

In [50]:
analysis(metric_df,sheet_title)